<a href="https://colab.research.google.com/github/sahiser19/ST-554-Project1-Sarah_Ryan_Heather/blob/Task-3---Heather/Task_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction

## Aims and Goals

The main goal of this portion of the data analysis is to determine which model of regression is best for our data set.

To do this we will predict the benzene concentration given a variety of variables. We are going to try and then test two different regression models.

The first model is a single linear regression model which uses the concentration of carbon dioxide in the air to predict the concentration of benzene in the air.

The second model is a multipule linear regression model which looks at carbon dioxide concentration, temperature, relative humidity, and absolute humidity to predict the concentration of benzene in the air.

One complicating factor of this data is that it is run over a period of time. Since it is gathered sequentially it makes sense to analyze the data sequentially. This is due to the fact that two days in a row may have very similar values. To do this we will have to write our own cross validation function.

This report will contain an initial fit of the data. The cross validation function written along with results and interpritations. Finally, a final fit will be done.

## Read in Data

All coding starts with reading in and cleaning up the data. This data was gathered using

In [42]:
!pip install ucimlrepo
import ucimlrepo as uci
import numpy as np
import pandas as pd
air_quality = uci.fetch_ucirepo(id=360)
air_quality.data.features

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,4/4/2005,10:00:00,3.1,1314,-200,13.5,1101,472,539,190,1374,1729,21.9,29.3,0.7568
9353,4/4/2005,11:00:00,2.4,1163,-200,11.4,1027,353,604,179,1264,1269,24.3,23.7,0.7119
9354,4/4/2005,12:00:00,2.4,1142,-200,12.4,1063,293,603,175,1241,1092,26.9,18.3,0.6406
9355,4/4/2005,13:00:00,2.1,1003,-200,9.5,961,235,702,156,1041,770,28.3,13.5,0.5139


## Clean Data

The first thing I am going to do is create a dataframe where only the variables I want to reference for this exsist and replace all of the missing data values (inidcated by the value being -200) with NaN so that I can remove those rows. The sum shows that with my CO I will be removing a lot of rows so that is something to look out for. I kept; CO (carbon monoxide), C6H6 (benzene), T(temperature), RH(realtive humididty), and AH(absolute humidity).

In [49]:
air_mod = air_quality.data.features.drop(columns = air_quality.data.features.columns[[1, 3,4,6,7,8,9,10,11]])
(air_mod == -200).sum()

,0
Date,0
CO(GT),1683
C6H6(GT),366
T,366
RH,366
AH,366


In [54]:
air_mod = air_quality.data.features.drop(columns = air_quality.data.features.columns[[1, 3,4,6,7,8,9,10,11]])
air_mod.replace(-200, np.nan, inplace= True)
air_mod.dropna(inplace=True)
air_mod

,Date,CO(GT),C6H6(GT),T,RH,AH
0,3/10/2004,2.6,11.9,13.6,48.9,0.7578
1,3/10/2004,2.0,9.4,13.3,47.7,0.7255
2,3/10/2004,2.2,9.0,11.9,54.0,0.7502
3,3/10/2004,2.2,9.2,11.0,60.0,0.7867
4,3/10/2004,1.6,6.5,11.2,59.6,0.7888
...,...,...,...,...,...,...
9352,4/4/2005,3.1,13.5,21.9,29.3,0.7568
9353,4/4/2005,2.4,11.4,24.3,23.7,0.7119
9354,4/4/2005,2.4,12.4,26.9,18.3,0.6406
9355,4/4/2005,2.1,9.5,28.3,13.5,0.5139


Next is to create a dataframe that is organized by date with the values being the mean values of each variable for each the dates. I do this using .groupby

In [55]:
air_test = air_mod.groupby('Date').mean()
air_test

,CO(GT),C6H6(GT),T,RH,AH
Date,,,,,
1/1/2005,2.134783,7.313043,6.813043,51.260870,0.501643
1/10/2005,2.127273,13.463636,13.377273,68.413636,1.044486
1/11/2005,2.812500,13.779167,12.779167,64.104167,0.941413
1/12/2005,3.273913,15.817391,12.021739,65.443478,0.904865
1/13/2005,2.679167,12.495833,9.991667,69.566667,0.847225
...,...,...,...,...,...
9/5/2004,1.304167,5.712500,29.204167,27.237500,1.057429
9/6/2004,1.421739,6.278261,26.752174,35.934783,1.219478
9/7/2004,1.890909,9.700000,27.618182,33.318182,1.154682


Then I added a days column to make my indexing easier.

In [56]:
air_test['Day']=range(1,348)
air_test

,CO(GT),C6H6(GT),T,RH,AH,Day
Date,,,,,,
1/1/2005,2.134783,7.313043,6.813043,51.260870,0.501643,1
1/10/2005,2.127273,13.463636,13.377273,68.413636,1.044486,2
1/11/2005,2.812500,13.779167,12.779167,64.104167,0.941413,3
1/12/2005,3.273913,15.817391,12.021739,65.443478,0.904865,4
1/13/2005,2.679167,12.495833,9.991667,69.566667,0.847225,5
...,...,...,...,...,...,...
9/5/2004,1.304167,5.712500,29.204167,27.237500,1.057429,343
9/6/2004,1.421739,6.278261,26.752174,35.934783,1.219478,344
9/7/2004,1.890909,9.700000,27.618182,33.318182,1.154682,345


#Initial Fit

#Cross Validation

##Helper Function

A helper function that calculates only one MSE for one set of training and test data was created to factillitate the later cross-validation function.

In [ ]:
def MSE()

##Cross Validation Function

# Final Fit